In [1]:
from pennylane import numpy as np
import pandas as pd
import pennylane as qml

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import time
import sys, os

utils_directory = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data_preparation/ids')
sys.path.append(utils_directory)

from utils.flow_windows import Flows
from utils.probabilities import Probabilities
from utils.density_matrices import DensityMatrix

In [ ]:
data_file_path = 'datasets/unsw-nb15/'
timing_log_file = 'datasets/unsw-nb15/timing_log.txt'

In [3]:
RATE = 0.0001

In [4]:
def log_to_file(message, filename):
    with open(filename, 'a') as file:  # 'a' opens the file for appending
        file.write(message + "\n")

In [ ]:
raw_df = pd.read_csv(f'{data_file_path}unsw_processed_nmf.csv')
raw_df

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,protocol,t_delta,label
0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.138385,0.000000,0.012689,0.0,0.000000,0.011432,0.017710,others,0.000010,analysis
1,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.110393,0.000000,0.011628,0.0,0.000000,0.013364,0.008059,others,0.000005,analysis
2,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000009,0.000000,0.016255,0.0,0.000000,0.021256,0.035198,others,0.000008,analysis
3,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.109939,0.000000,0.011966,0.0,0.000000,0.013561,0.007821,pipe,0.000004,analysis
4,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000009,0.000000,0.016255,0.0,0.000000,0.021256,0.035198,others,0.000003,analysis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79876,0.0,0.015327,0.0,0.0,0.000000,0.005023,0.0,0.0,0.0,0.000000,...,0.031808,0.097870,0.000000,0.0,0.062786,0.026826,0.045825,tcp,0.000009,worms
79877,0.0,0.000000,0.0,0.0,0.000019,0.002751,0.0,0.0,0.0,0.000000,...,0.047085,0.084416,0.028820,0.0,0.000764,0.046952,0.051968,tcp,0.000007,worms
79878,0.0,0.013966,0.0,0.0,0.000158,0.001632,0.0,0.0,0.0,0.000000,...,0.040241,0.138970,0.023965,0.0,0.069022,0.040768,0.044637,tcp,0.000004,worms
79879,0.0,0.015326,0.0,0.0,0.000000,0.005019,0.0,0.0,0.0,0.000000,...,0.032334,0.097920,0.000000,0.0,0.063651,0.026719,0.045712,tcp,0.000009,worms


In [6]:
base_time = 1.540446e+09
raw_df['cumulative_t_delta'] = raw_df['t_delta'].cumsum()
raw_df['frame.time_epoch'] = base_time + raw_df['cumulative_t_delta']

In [7]:
for col in raw_df.columns:
    if col.isnumeric():
        continue
    print(col, raw_df[col].unique())

protocol ['others' 'pipe' 'ipip' 'rdp' 'micp' 'crtp' 'snp' 'vmtp' 'tcp' 'etherip'
 'leaf-2' 'gre' 'ax.25' 'nvp' 'sctp' 'sep' 'unas' 'iplt' 'gmtp'
 'secure-vmtp' 'sun-nd' 'ib' 'swipe' 'sps' 'hmp' 'rsvp' 'egp' 'ospf'
 'mobile' 'ipv6' 'fire' 'pim' 'ggp' 'sccopmce' 'dgp' 'emcon' 'crudp' 'fc'
 'udp' 'arp' 'icmp']
t_delta [1.0000e-05 5.0000e-06 8.0000e-06 4.0000e-06 3.0000e-06 6.0000e-06
 9.0000e-06 1.1000e-05 2.0000e-06 1.2000e-05 7.0000e-06 1.0000e-06
 4.9000e-05 2.4000e-05 2.1000e-04 1.9000e-05 1.8000e-05 3.4000e-05
 1.5000e-05 3.2000e-05 5.5200e-04 2.7000e-05 8.2200e-04 1.3000e-05
 2.5500e-04 6.9000e-04 1.6000e-05 2.3870e-03 3.1440e-03 2.6100e-04
 1.6910e-02 2.2600e-04 1.7000e-05 6.0230e-03 8.0500e-04 1.2800e-03
 7.5300e-04 1.4000e-05 4.2300e-04 1.6342e-02 2.5820e-03 2.9000e-05
 0.0000e+00 2.0000e-05 2.1000e-05 3.3000e-05 3.1000e-05 2.2100e-04
 2.3000e-05 5.0160e-03 6.4600e-04 1.1400e-04 2.8000e-05 5.7940e-03
 2.5000e-05 3.7000e-05 3.6000e-05 1.3900e-04 3.0000e-05 3.5000e-05
 2.1260e-03 

In [8]:
raw_df['label'].value_counts()

label
normal            21000
generic           17580
exploits          13992
fuzzers           12722
reconnaissance     7562
dos                3397
backdoor           1239
analysis           1208
shellcode          1088
worms                93
Name: count, dtype: int64

# Raw

In [44]:
raw_df_processed = raw_df.copy()

In [45]:
def one_hot_encoding(df, column):
    df = pd.concat([df, pd.get_dummies(df[column], prefix=column)], axis=1)
    df.drop(column, axis=1, inplace=True, errors='ignore')
    return df

In [ ]:
raw_df_processed = one_hot_encoding(raw_df_processed, 'protocol')

In [47]:
raw_df_processed = raw_df_processed.drop(columns=['t_delta'])

In [ ]:
raw_df_processed.to_csv(f"{data_file_path}processed_rawcsv", index=False)

# DM/STATS Combined

In [15]:
def df_flows_probs(df, rate):
    flow_method = "time"

    prob_bases = ["protocol"]
    prob_base = "protocol"
    prob_method = "bayesian"

    flows = Flows(df=df, method=flow_method, flow_duration=rate)
    df_dm_flows = flows.main()

    probs = Probabilities(
        df_flows=df_dm_flows, bases=prob_bases, base=prob_base, method=prob_method
    )
    df_dm_probs = probs.main()

    return df_dm_probs

def df_subset_labels(df, rate):
    labels = df['label'].unique()
    df_subsets = []
    flow_counter = 0
    for label in labels:
        df_subset = df[df['label'] == label]
        df_subset = df_flows_probs(df_subset, rate=rate)
        if len(df_subset) > 1:
            if not flow_counter == 0:
                df_subset.loc[:, 'flow_window'] += flow_counter
            flow_counter += df_subset['flow_window'].max()
            df_subsets.append((label, df_subset))
        else:
            df_subsets.append((label, None))
    return df_subsets


In [16]:
start_time = time.time()

df_subsets = df_subset_labels(raw_df, rate=RATE)

dm_stat_processing_time = time.time() - start_time


/Users/larryhh/Documents/PhD/Projects/quantum_encodings_in_classical_settings/src/data_preparation/ids/utils/flow_windows.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["frame.date_time"] = self.df["frame.time_epoch"].apply(
/Users/larryhh/Documents/PhD/Projects/quantum_encodings_in_classical_settings/src/data_preparation/ids/utils/flow_windows.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["frame.date_time"] = self.df["frame.time_epoch"].apply(
/Users/larryhh/Documents/PhD/Proje

In [17]:
df_subsets[1][1]

,0,1,2,3,4,5,6,7,8,9,...,49,protocol,t_delta,label,cumulative_t_delta,frame.time_epoch,frame.date_time,flow_window,dm_prob,dm_prob_softmax
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.035198,others,0.000003,backdoor,0.009489,1.540446e+09,2018-10-25 13:40:00.009489,96,0.959635,0.100634
1,0.0,0.0,0.0,0.0,0.0,0.003148,0.000000,0.0,0.0,0.0,...,0.045088,tcp,0.000004,backdoor,0.009493,1.540446e+09,2018-10-25 13:40:00.009493,96,0.017090,0.039210
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.008900,others,0.000008,backdoor,0.009501,1.540446e+09,2018-10-25 13:40:00.009501,96,0.959635,0.100634
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.035318,others,0.000005,backdoor,0.009506,1.540446e+09,2018-10-25 13:40:00.009506,96,0.959635,0.100634
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.035313,others,0.000024,backdoor,0.009530,1.540446e+09,2018-10-25 13:40:00.009530,96,0.959635,0.100634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,tcp,0.000008,backdoor,0.071166,1.540446e+09,2018-10-25 13:40:00.071166,712,0.022364,0.043569
1235,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.008461,others,0.000008,backdoor,0.071174,1.540446e+09,2018-10-25 13:40:00.071174,712,0.941853,0.109272
1236,0.0,0.0,0.0,0.0,0.0,0.000027,0.000000,0.0,0.0,0.0,...,0.023588,gre,0.000012,backdoor,0.071186,1.540446e+09,2018-10-25 13:40:00.071186,712,0.002769,0.042724
1237,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.008269,others,0.000010,backdoor,0.071196,1.540446e+09,2018-10-25 13:40:00.071196,713,0.904294,0.691809


In [18]:
df_combined = [df_subset[1] for df_subset in df_subsets if df_subset[1] is not None]
df_combined = pd.concat(df_combined, axis=0)

In [ ]:
df_combined.to_csv(f"{data_file_path}DM_STATS_COMBINED.csv", index=False)

# DM

In [19]:
df_dm = df_combined.copy()
df_dm

,0,1,2,3,4,5,6,7,8,9,...,49,protocol,t_delta,label,cumulative_t_delta,frame.time_epoch,frame.date_time,flow_window,dm_prob,dm_prob_softmax
0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.017710,others,0.000010,analysis,0.000010,1.540446e+09,2018-10-25 13:40:00.000010,1,0.997250,0.074203
1,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.008059,others,0.000005,analysis,0.000015,1.540446e+09,2018-10-25 13:40:00.000015,1,0.997250,0.074203
2,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.035198,others,0.000008,analysis,0.000023,1.540446e+09,2018-10-25 13:40:00.000023,1,0.997250,0.074203
3,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.007821,pipe,0.000004,analysis,0.000027,1.540446e+09,2018-10-25 13:40:00.000027,1,0.000688,0.027392
4,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.035198,others,0.000003,analysis,0.000030,1.540446e+09,2018-10-25 13:40:00.000030,1,0.997250,0.074203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.015327,0.0,0.0,0.000000,0.005023,0.0,0.0,0.0,0.000000,...,0.045825,tcp,0.000009,worms,0.845373,1.540446e+09,2018-10-25 13:40:00.845373,254608,0.888889,0.119356
89,0.0,0.000000,0.0,0.0,0.000019,0.002751,0.0,0.0,0.0,0.000000,...,0.051968,tcp,0.000007,worms,0.845380,1.540446e+09,2018-10-25 13:40:00.845380,254608,0.888889,0.119356
90,0.0,0.013966,0.0,0.0,0.000158,0.001632,0.0,0.0,0.0,0.000000,...,0.044637,tcp,0.000004,worms,0.845384,1.540446e+09,2018-10-25 13:40:00.845384,254608,0.888889,0.119356
91,0.0,0.015326,0.0,0.0,0.000000,0.005019,0.0,0.0,0.0,0.000000,...,0.045712,tcp,0.000009,worms,0.845393,1.540446e+09,2018-10-25 13:40:00.845393,254608,0.888889,0.119356


Commented is using protocol as probs

In [ ]:
time_columns = ["frame.time_epoch", "frame.date_time", "flow_window", 't_delta', 'cumulative_t_delta']
probability_columns = ["dm_prob", "dm_prob_softmax"]
other_columns = ["protocol"]
non_feature_columns = time_columns + probability_columns + other_columns

for col in [c for c in df_dm.columns if c not in non_feature_columns + ['label']]:
    df_dm[col] = pd.to_numeric(df_dm[col], errors='ignore')

import category_encoders as ce

def binary_encoding(df, column):
    encoder = ce.BinaryEncoder(cols=[column])
    df_encoded = encoder.fit_transform(df)
    return df_encoded

def remove_protocol(df, protocol):
    df = df.drop(columns=[col for col in df.columns if col.startswith(protocol)])
    return df

df_dm = remove_protocol(df_dm, 'protocol')

time_columns = ["frame.time_epoch", "frame.date_time", "flow_window", 't_delta', 'cumulative_t_delta']
probability_columns = ["dm_prob", "dm_prob_softmax"]
non_feature_columns = time_columns + probability_columns

for col in [c for c in df_dm.columns if c not in non_feature_columns + ['label']]:
    df_dm[col] = pd.to_numeric(df_dm[col], errors='ignore')

In [21]:
def collapse_labels(y):
    collapsed_labels = []
    for labels in y:
        collapsed_label = []
        for label in labels:
            collapsed_label.append(label.item())
        collapsed_label = max(set(collapsed_label), key=collapsed_label.count)
        collapsed_labels.append(collapsed_label)
    return collapsed_labels

In [ ]:
start_time = time.time()

dm = DensityMatrix(df_probs=df_dm, non_feature_columns=non_feature_columns, prob_mode=None, seed=0)
dm.construct_flows()
X, y = dm.construct_ml_dataset()

X = pd.DataFrame(X.reshape(X.shape[0], -1))
y = collapse_labels(y)
y = pd.DataFrame(y, columns=["label"])

log_message = f"DM.csv: {time.time() - start_time}"
log_to_file(log_message, timing_log_file)

df_dm = pd.concat([X, y], axis=1)
df_dm.to_csv(f"{data_file_path}processed_DM_{RATE}.csv", index=False)


# STATS

In [ ]:
df_stats = df_combined.copy()

In [25]:
df_stats

,0,1,2,3,4,5,6,7,8,9,...,protocol_secure-vmtp,protocol_sep,protocol_snp,protocol_sps,protocol_sun-nd,protocol_swipe,protocol_tcp,protocol_udp,protocol_unas,protocol_vmtp
0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
1,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
2,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
3,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
4,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.015327,0.0,0.0,0.000000,0.005023,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,True,False,False,False
89,0.0,0.000000,0.0,0.0,0.000019,0.002751,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,True,False,False,False
90,0.0,0.013966,0.0,0.0,0.000158,0.001632,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,True,False,False,False
91,0.0,0.015326,0.0,0.0,0.000000,0.005019,0.0,0.0,0.0,0.000000,...,False,False,False,False,False,False,True,False,False,False


In [ ]:
def process_group(group):
    time_columns = ["frame.time_epoch", "frame.date_time", "flow_window", 't_delta', 'cumulative_t_delta']
    probability_columns = ["dm_prob", "dm_prob_softmax"]
    non_feature_columns = time_columns + probability_columns

    selected_columns = [col for col in group.columns if col not in non_feature_columns]
    group = group[selected_columns]
    
    # Use pop to remove 'label' column and store it in a variable
    label_column = group.pop('label')

    stats = ['min', 'max', 'median', 'mean', 'var']
    stats_df = group.agg(stats)

    flat_stats = stats_df.to_numpy().flatten()

    # Generate column names
    col_names = []
    for stat in stats:
        col_names.extend([f'{col}_{stat}' for col in group.columns])

    all_stats = np.concatenate([flat_stats])
    # Convert to pandas DataFrame for easy handling
    stats_series = pd.Series(all_stats, index=col_names)

    # Insert 'label' column back into the DataFrame
    group.insert(len(group.columns), 'label', label_column)

    return stats_series


def stats_features(df):
    grouped = df.groupby("flow_window")
    features = grouped.apply(process_group)
    labels_filenames_folds = grouped.apply(lambda g: pd.Series({'label': g['label'].iloc[0]}))
    features_combined = features.join(labels_filenames_folds)
    return features_combined

In [38]:
def compute_stats(df):
    labels = df['label'].unique()
    df_subsets = []
    for label in labels:
        df_subset = df[df['label'] == label]
        df_subset = stats_features(df_subset)
        # df_subset = df_subset.dropna(axis=1)
        df_subsets.append(df_subset)
    df = pd.concat(df_subsets, axis=0)
    df = df.fillna(0)
    return df

In [39]:
df_stats['label'].value_counts().sum()

79881

In [ ]:
start_time = time.time()
df_stats = compute_stats(df_stats)

log_message = f"STATS.csv: {time.time() - start_time}"
log_to_file(log_message, timing_log_file)

df_stats.to_csv(f"{data_file_path}/processed_STATS_{RATE}.csv", index=False)

In [30]:
df_stats.isna().sum().sum()

0

In [31]:
df_stats['label'].value_counts()

label
generic           1242
exploits          1019
fuzzers            920
normal             911
reconnaissance     565
dos                258
backdoor           101
shellcode           84
analysis            82
worms                8
Name: count, dtype: int64

In [33]:
df_stats

,0_min,1_min,2_min,3_min,4_min,5_min,6_min,7_min,8_min,9_min,...,protocol_sep_var,protocol_snp_var,protocol_sps_var,protocol_sun-nd_var,protocol_swipe_var,protocol_tcp_var,protocol_udp_var,protocol_unas_var,protocol_vmtp_var,label
flow_window,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,analysis
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.066667,0.0,0.0,0.0,0.171429,0.000000,0.000000,0.066667,analysis
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,analysis
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0625,0.062500,0.0,0.0,0.0,0.116667,0.000000,0.000000,0.000000,analysis
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.000000,0.0,0.0,0.0,0.058824,0.000000,0.154412,0.058824,analysis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.000000,0.0,0.0,0.0,0.204545,0.204545,0.000000,0.000000,worms
254605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.000000,0.0,0.0,0.0,0.076923,0.076923,0.000000,0.000000,worms
254606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.000000,0.0,0.0,0.0,0.263736,0.263736,0.000000,0.000000,worms
